In [117]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, classification_report
from sklearn.multioutput import MultiOutputClassifier



In [118]:
db = 'urinary_unfiltered'
region = 'V3V4' # 'dna_seq' or 'V3V4'. Will only apply to training sequences

In [ ]:
################################ Datas and variables
# paths to reference files
ref_data = '../datasets/train_sets/' + db + '.csv'


# samples sequences too classify import
unknown_df = pd.read_csv('/home/marthe/Documents/DS/projet/local/pre_processed_data/thresh_17/dna_sequences_17.csv')


# levels used for taxonomy
taxonomy_levels = ['domain', 'phylum', 'class', 'order', 'family', 'genus', 'species']





In [120]:
################################ Functions

# sequences segmentation into k-mers, returns list
def generate_kmers(sequence, k):
    """Generate k-mers from a DNA sequence."""
    kmers = []
    for i in range(len(sequence) - k + 1):
        kmers.append(sequence[i:i+k])
    kmers = ' '.join(kmers)
    return kmers

# results evaluation
def print_scores(y_true, y_pred, levels_list, score_file):
    
    for level in levels_list:
        content = ''
        content = content \
            + level \
            + '\n' \
            +  '-' * len(level) \
            + '   accuracy : ' \
            + str(accuracy_score(y_true[level], y_pred[level])) \
            + '   precision :' \
            + str(precision_score(y_true[level], y_pred[level], average = 'weighted', zero_division = np.nan)) \
            + '   score f1 :' \
            + str(f1_score(y_true[level], y_pred[level], average = 'weighted', zero_division = np.nan)) \
            + '\n' \
            + classification_report(y_true[level], y_pred[level], zero_division = np.nan) \
            + '\n\n'

        

        with open(score_file, mode = 'a') as file:
            file.write(content)

        print(content)    
    

# get predictions and probabilities for each sequence
# replace predictions with low probability with 'unclassified'
# returns lists
def pred_and_proba(preds, probas, idx, thresh):
    pred_vec = []
    proba_vec = [max(probas[k][idx]) for k in range(len(taxonomy_levels))]
    for level, proba in enumerate(proba_vec):
        if proba > thresh[level]:
            pred_vec.append(preds[idx][level])
        else:
            pred_vec.extend(["unclassified"] * (len(taxonomy_levels) - level)) # once a level is set to 'unclassified', lower levels are too
            break
        
    return pred_vec, proba_vec


# print 'unclassified' rate for each taxonomy level of a results df
def unclassified_rate(df, score_file):
    for level in taxonomy_levels:
        unclassified_count = df[level].value_counts(normalize = True)
        if 'unclassified' in unclassified_count.index:
            unclassified_rate = unclassified_count['unclassified'] * 100
        else:
            unclassified_rate = 0
        with open(score_file, mode = 'a') as file:
            file.write("Taux de 'unclassified' au rang %s: %.2f%%\n"%(level, unclassified_rate))
        print("Taux de 'unclassified' au rang %s: %.2f%%"%(level, unclassified_rate))
        
    print('\n')











In [121]:
################################ Sequences vectorization

# reference file importation and pre-process
known_df = pd.read_csv(ref_data)
known_df.fillna('undefined', inplace = True)
known_df.drop_duplicates(region, inplace = True)

# train_df sequences segmentation
known_df['kmers'] = known_df[region].apply(lambda x: generate_kmers(x, 7))

# train and test data
X = known_df['kmers']
y = known_df[taxonomy_levels]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1234)

# Features vectorization
vectorizer = CountVectorizer()

X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
################################ Model training


# params for cross validation
params = {'estimator__alpha' : [1, 0.1, 0.05, 0.01, 0.005, 0.001]}

# model
model = MultinomialNB()

moc_cv_model = GridSearchCV(MultiOutputClassifier(model), param_grid = params)

moc_cv_model.fit(X_train_vectorized, y_train)

# model saving
joblib.dump(moc_cv_model, '../models/NB_oneset' + db + '_' + region + '.joblib')

        

/home/marthe/miniconda3/envs/microbiote/lib/python3.11/site-packages/sklearn/model_selection/_split.py:2734: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  and (type_of_target(y, input_name="y") in ("binary", "multiclass"))
/home/marthe/miniconda3/envs/microbiote/lib/python3.11/site-packages/sklearn/utils/multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_type = type_of_target(y, input_name="y")
/home/marthe/miniconda3/envs/microbiote/lib/python3.11/site-packages/sklearn/utils/multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_type = type_of_target(y, input_name="y")
/home/marthe/miniconda3/envs/microbiote/lib/pyt

['/home/marthe/Documents/DS/projet/local/frogsdays/NB_oneseturinary_unfiltered_V3V4.joblib']

In [ ]:
################################ Model prediction

moc_cv_model = joblib.load('../models/NB_oneset' + db + '_' + region + '.joblib')

print('DONNÉES DE RÉFÉRENCE : ', db)
print('Restriction des séquences : ', region)
print('----------------------------------------------\n\n')


# predictions for test_df
y_pred_moc_cv = moc_cv_model.predict(X_test_vectorized)
y_pred_moc_cv_proba = moc_cv_model.predict_proba(X_test_vectorized)

# results agregation in dataframe, with set thresholds to get rid of low probability predictions
proba_columns = ['proba_'+level for level in taxonomy_levels]
pred_results = pd.DataFrame(columns = taxonomy_levels + proba_columns, index = X_test.index)

thresh = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]

for k in range(pred_results.shape[0]):
    pred_vec_k, proba_vec_k = pred_and_proba(y_pred_moc_cv, y_pred_moc_cv_proba, k, thresh)
    pred_results.loc[k] = pred_vec_k + proba_vec_k

# test predictions saving
# pred_results_to_save = pd.concat([X_test['seq_id'], pred_results], axis = 1)
# pred_results_to_save.to_csv('../models/NB_test_' + ref + '_' + region + '.csv', index = False)


# score_file = '../scores/NB/scoresNB_'+ ref + '_' + region + '.txt'

# test predictions evaluation
# with open(score_file, mode = 'a') as file:
#     file.write('DONNÉES DE RÉFÉRENCE : ' + ref + '\n')
#     file.write('Restriction des séquences : ' + region + '\n')
#     file.write('----------------------------------------------\n\n')
# unclassified_rate(pred_results, score_file)
# print_scores(y_test, pred_results, taxonomy_levels, score_file)




DONNÉES DE RÉFÉRENCE :  urinary_unfiltered
Restriction des séquences :  V3V4
----------------------------------------------




In [ ]:
############################### Samples sequences classification
# sequences segmentation
unknown_df['kmers'] = unknown_df['dna_seq'].apply(lambda x: generate_kmers(x, 7))

# features vectorization
X_unknown = unknown_df['kmers']
X_unknown_vectorized = vectorizer.transform(X_unknown)

# prediction
samples_prediction = moc_cv_model.predict(X_unknown_vectorized)
samples_proba = moc_cv_model.predict_proba(X_unknown_vectorized)

# results agregation in dataframe, with set thresholds to get rid of low probability predictions
proba_columns = ['proba_'+level for level in taxonomy_levels]
samples_results = pd.DataFrame(columns = ['seq_id'] + taxonomy_levels + proba_columns, index = X_test.index)

thresh = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]

for k in range(samples_prediction.shape[0]):
    seq_id = [unknown_df['seq_id'][k]]
    pred_vec_k, proba_vec_k = pred_and_proba(samples_prediction, samples_proba, k, thresh)
    item = seq_id + pred_vec_k + proba_vec_k
    samples_results.loc[k] = item

# samples prediction saving
samples_results.to_csv('../preds/samples/NB_samples_' + db + '_' + region + '.csv', index = False)



In [125]:
preds = pd.DataFrame(y_pred_moc_cv, columns = taxonomy_levels, index = X_test.index)
preds

,domain,phylum,class,order,family,genus,species
3568,Bacteria,Bacillota,Bacilli,Bacillales,Staphylococcaceae,Staphylococcus,Staphylococcus schleiferi
3069,Bacteria,Pseudomonadota,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Aquipseudomonas,Aquipseudomonas alcaligenes
2629,Bacteria,Pseudomonadota,Betaproteobacteria,Neisseriales,Neisseriaceae,Neisseria,Neisseria sicca
38,Bacteria,Actinomycetota,Actinomycetes,Bifidobacteriales,Bifidobacteriaceae,Bifidobacterium,Bifidobacterium dentium
1913,Bacteria,Actinomycetota,Actinomycetes,Propionibacteriales,Nocardioidaceae,Flindersiella,Flindersiella endophytica
...,...,...,...,...,...,...,...
1927,Bacteria,Fusobacteriota,Fusobacteriia,Fusobacteriales,Fusobacteriaceae,Fusobacterium,Fusobacterium nucleatum
3298,Bacteria,Bacillota,Clostridia,Eubacteriales,Oscillospiraceae,Ruminococcus,Ruminococcus bromii
616,Bacteria,Pseudomonadota,Betaproteobacteria,Burkholderiales,Alcaligenaceae,Advenella,Advenella mimigardefordensis
3902,Bacteria,Bacillota,Negativicutes,Veillonellales,Veillonellaceae,Veillonella,Veillonella montpellierensis


In [ ]:
score_file = '../scores/recap_scores.csv'

with open(score_file, mode = 'a') as file:
    for level in taxonomy_levels[-3:]:
        true_labels = y_test[level]
        pred_labels = preds[level]
        print(level)
        accuracy = accuracy_score(true_labels, pred_labels)
        print(accuracy)
        precision = precision_score(true_labels, pred_labels, average = 'weighted', zero_division = np.nan)
        print(precision)
        f1 = f1_score(true_labels, pred_labels, average = 'weighted', zero_division = np.nan)
        print(f1)
        
        file.write('NB oneset, {}, {}, {}, {}, {}, {}\n'.format(db, region, level, accuracy, precision, f1))


family
0.9696202531645569
0.9784856929895098
0.9698202277882058
genus
0.8936708860759494
0.9474701079031786
0.8961899624113475
species
0.759493670886076
0.9140873015873016
0.7484267631103074


/home/marthe/miniconda3/envs/microbiote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
/home/marthe/miniconda3/envs/microbiote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")
/home/marthe/miniconda3/envs/microbiote/lib/python3.11/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
/home/marthe/miniconda